### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data, school_data, how="inner", on=["school_name", "school_name"])


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#summary calculations
school_count = school_data_complete_df["school_name"].nunique()
student_count = school_data_complete_df["student_name"].count()
math_average = school_data_complete_df['math_score'].mean()
read_average = school_data_complete_df['reading_score'].mean()

#total budget
unique_budget = school_data_complete_df['budget'].unique()
total_budget = unique_budget.sum()
total_buget = str(total_budget)

# % passing math
pass_math_df = school_data_complete_df.loc[school_data_complete_df["math_score"] >= 70, :    ]
pass_math_count = pass_math_df['math_score'].count()
math_pass = (pass_math_count / student_count * 100).round(6)

# % passing reading
pass_read_df = school_data_complete_df.loc[school_data_complete_df['reading_score'] >= 70, :   ]
pass_read_count = pass_read_df['reading_score'].count()
read_pass = (pass_read_count / student_count * 100).round(6)

# % passing reading and math
pass_read_math_df = school_data_complete_df.loc[(school_data_complete_df['math_score'] >= 70) & 
                                                (school_data_complete_df['reading_score'] >= 70), :  ]
pass_read_math_count = pass_read_math_df['reading_score'].count()
pass_read_math = (pass_read_math_count / student_count * 100).round(6)

In [7]:
#creating summary df
district_summary_df = pd.DataFrame({'Total Schools': [school_count], 
                                   'Total Student': student_count,
                                   'Total Budget': total_buget,
                                    'Average Math Score': math_average,
                                    'Average Reading Score': read_average,
                                    '% Passing Math': math_pass,
                                    '% Passing Reading': read_pass,
                                    '% Overall Passing': pass_read_math,
                                   })

district_summary_df['Total Student'] = district_summary_df['Total Student'].astype(int).map("{:,}".format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].astype(int).map("${:,.2f}".format)

district_summary_df

,Total Schools,Total Student,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326



## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [8]:
school_split_df = school_data_complete_df

In [9]:
#create pass and fail math score bins
bins = [0, 69.9, 100]

group_labels = [0 , 1]

school_split_df["Math Pass/Fail"] = pd.cut(school_split_df['math_score'], bins, labels=group_labels)
school_split_df["Reading Pass/Fail"] = pd.cut(school_split_df['reading_score'], bins, labels=group_labels)

In [10]:
#casting 
school_split_df["Math Pass/Fail"] = school_split_df["Math Pass/Fail"].astype("int")
school_split_df["Reading Pass/Fail"] = school_split_df["Reading Pass/Fail"].astype("int")

In [11]:
# overall passing
school_split_df["Overall Pass/Fail"] = school_split_df["Reading Pass/Fail"] + school_split_df["Math Pass/Fail"]

In [12]:
#create df to filter for overall passing
overall_pass_df = school_split_df.loc[school_split_df["Overall Pass/Fail"] >= 2, : ]

In [13]:
#group overall pass by school
group_pass_df = overall_pass_df.groupby(["school_name"])

In [14]:
#group by school_name
group_df = school_split_df.groupby(["school_name"])

In [16]:
#creating columns for summary data table
reading_score_average = group_df["reading_score"].mean()
math_score_average = group_df['math_score'].mean()
total_students = group_df['Student ID'].count() 
percent_math_pass = (group_df["Math Pass/Fail"].sum() / group_df['Student ID'].count()) * 100
percent_read_pass = (group_df["Reading Pass/Fail"].sum() / group_df['Student ID'].count()) * 100
overall_pass = (group_pass_df['Overall Pass/Fail'].sum() / group_df['Student ID'].count())*100 / 2
budget_school = (group_df["budget"].sum()/group_df['Student ID'].count())
budget_student = budget_school / total_students
school_type = group_df["type"].min()


In [17]:
#formatting
budget_school = budget_school.astype(float).map("${:,.2f}".format)
budget_student = budget_student.astype(float).map("${:,.2f}".format)

In [18]:
#summary df
school_summary_df = pd.DataFrame({'School Type': school_type,
                                  'Total Students': total_students,
                                  'Total School Budget': budget_school,
                                  'Per Student Budget': budget_student,
                                 'Average Reading Score': reading_score_average,
                                 'Average Math Score': math_score_average,
                                 'Total Students': total_students,
                                 '% Passing Math': percent_math_pass,
                                 '% Passing Reading': percent_read_pass,
                                 '% Overall Passing': overall_pass})

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,81.033963,77.048432,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.975780,83.061895,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,80.746258,77.102592,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.816757,83.351499,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,80.934412,77.289752,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.814988,83.803279,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,81.182722,76.629414,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,80.966394,77.072464,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [19]:
top_school_df = school_summary_df.sort_values("% Overall Passing", ascending=False)
top_school_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.975780,83.061895,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.848930,83.418349,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.816757,83.351499,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.989488,83.274201,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,84.044699,83.839917,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [20]:
top_school_df = school_summary_df.sort_values("% Overall Passing", ascending=True)
top_school_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,80.744686,76.842711,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,81.182722,76.629414,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,80.934412,77.289752,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,80.966394,77.072464,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [22]:
#filter math scores by grade can create df
math_by_grade_df = school_data_complete_df[['math_score', 'grade', 'school_name']]

math_9_df = math_by_grade_df.loc[math_by_grade_df['grade'] == '9th', : ]
math_10_df = math_by_grade_df.loc[math_by_grade_df['grade'] == '10th', : ]
math_11_df = math_by_grade_df.loc[math_by_grade_df['grade'] == '11th', : ]
math_12_df = math_by_grade_df.loc[math_by_grade_df['grade'] == '12th', : ]

math_9 = math_9_df.groupby(['school_name']).mean()["math_score"]
math_10 = math_10_df.groupby(['school_name']).mean()["math_score"]
math_11 = math_11_df.groupby(['school_name']).mean()["math_score"]
math_12 = math_12_df.groupby(['school_name']).mean()["math_score"]


math_by_grade_df = pd.DataFrame({'9th': math_9,
                                '10th': math_10,
                                '11th': math_11,
                                '12th': math_12})

math_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [27]:
#filter reading scores by grade and create df
read_by_grade_df = school_data_complete_df[['reading_score', 'grade', 'school_name']]

read_9_df = read_by_grade_df.loc[read_by_grade_df['grade'] == '9th', : ]
read_10_df = read_by_grade_df.loc[read_by_grade_df['grade'] == '10th', : ]
read_11_df = read_by_grade_df.loc[read_by_grade_df['grade'] == '11th', : ]
read_12_df = read_by_grade_df.loc[read_by_grade_df['grade'] == '12th', : ]

read_9 = read_9_df.groupby(['school_name']).mean()["reading_score"]
read_10 = read_10_df.groupby(['school_name']).mean()["reading_score"]
read_11 = read_11_df.groupby(['school_name']).mean()["reading_score"]
read_12 = read_12_df.groupby(['school_name']).mean()["reading_score"]

read_by_grade_df = pd.DataFrame({'9th': read_9,
                                '10th': read_10,
                                '11th': read_11,
                                '12th': read_12})

read_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [25]:
read_by_grade_df = school_summary_df[["Average Math Score", "Average Reading Score", "Per Student Budget",
                                      "% Passing Reading", "% Passing Math", "% Overall Passing"]]

read_by_grade_df

,Average Math Score,Average Reading Score,Per Student Budget,% Passing Reading,% Passing Math,% Overall Passing
school_name,,,,,,
Bailey High School,77.048432,81.033963,$628.00,81.933280,66.680064,54.642283
Cabrera High School,83.061895,83.975780,$582.00,97.039828,94.133477,91.334769
Figueroa High School,76.711767,81.158020,$639.00,80.739234,65.988471,53.204476
Ford High School,77.102592,80.746258,$644.00,79.299014,68.309602,54.289887
Griffin High School,83.351499,83.816757,$625.00,97.138965,93.392371,90.599455
Hernandez High School,77.289752,80.934412,$652.00,80.862999,66.752967,53.527508
Holden High School,83.803279,83.814988,$581.00,96.252927,92.505855,89.227166
Huang High School,76.629414,81.182722,$655.00,81.316421,65.683922,53.513884
Johnson High School,77.072464,80.966394,$650.00,81.222432,66.057551,53.539172


In [26]:
#create pass and fail math score bins
bins = [0, 585, 630, 645, 680]

group_labels = [0 , 1]

school_split_df["Math Pass/Fail"] = pd.cut(school_split_df['math_score'], bins, labels=group_labels)

ValueError: Bin labels must be one fewer than the number of bin edges

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type